<a href="https://colab.research.google.com/github/asd93/a/blob/master/ch04.2_17101605.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/asd93/a/blob/master/ch04.2_17101605.ipynb)

## 신경망 학습
#### 학습 : 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것
#### 손실함수: 신경망 성능의 '나쁨'을 나타내는 지표 (평균제곱오차,교차엔트로피오차)
#### 손실함수의 결괏값을 가장 작게 만드는 가중치 매개변수를 찾는다.<br><br>손실함수의 값을 작게 만드는 기법 : 경사법(함수 기울기 활용)

### ①평균 제곱 오차
#### $E=\frac{1}{2}\sum_{k}(y_{k}-t_{k})^{2}$<br>$y_{k}$ : 신경망의 출력, $t_{k}$ : 정답 레이블, k : 데이터의 차원 수

In [0]:
import numpy as np
def mean_squared_error(y,t):
    return 0.5 * np.sum((y-t)**2)

In [0]:
#정답은 '2'
t=[0,0,1,0,0,0,0,0,0,0]

#예1:'2'일 확률이 가장 높다고 추정함(0.6)
y=[0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
print(mean_squared_error(np.array(y),np.array(t)))

#예2:'7'일 확률이 가장 높다고 추정함(0.6)
y=[0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
print(mean_squared_error(np.array(y),np.array(t)))

#예1의 손실함수 출력이 작고 정답 레이블과의 오차도 작으므로 정답에 가깝다.

0.09750000000000003
0.5975


### ②교차 엔트로피 오차
#### $E=-\sum_{k}t_{k}\log(y_{k})$<br>log : 밑이 e인 자연로그, $y_{k}$ : 신경망의 출력,  $t_{k}$ : 정답 레이블(원-핫인코딩)

In [0]:
def cross_entropy_error(y,t):
    delta=1e-7
    return -np.sum(t*np.log(y+delta))

In [0]:
#정답은 '2'
t=[0,0,1,0,0,0,0,0,0,0]

#예1:'2'일 확률이 가장 높다고 추정함(0.6)
y=[0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

#예2:'7'일 확률이 가장 높다고 추정함(0.6)
y=[0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

#예1의 오차 값이 더 작으므로 평균 제곱 오차의 판단과 일치.

0.510825457099338
2.302584092994546


#### 평균 손실 함수 
#### $E=-\frac{1}{N}\sum_{n}\sum_{k}t_{nk}\log(y_{nk})$
#### 미니배치 학습 : 신경망 학습에서 훈련 데이터로부터 일부만 골라 학습 수행 - 근사치 이용

In [0]:
import sys, os
sys.path.append(r"C:\Users\이유선\Desktop\'▽'\py\deep-learning-from-scratch-master")
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


In [0]:
#미니배치 데이터 꺼내기
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

np.random.choice(60000,10)

array([15427, 26248,  8372, 53331, 30742, 17201, 59425,  8541, 16257,
       45795])

In [0]:
#미니배치 배치 데이터 지원하는 교차 엔트로피 오차
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t=t.reshape(1,t.size)
        y=y.reshape(1,y.size)
        
    batch_size=y.shape[0]
    return -np.sum(t*np.log(y+1e-7)) / batch_size

#정답 레이블이 원-핫인코딩이 아니라 숫자 레이블일때
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t=t.reshape(1,t.size)
        y=y.reshape(1,y.size)
        
    batch_size=y.shape[0]
    return -np.sum(np.log(y[np.arange(batch.size), t] +1e-7)) / batch_size

#원-핫인코딩일때 t=0이면 오차도 0이므로 정답에 해당하는 신경망 출력만으로 오차 계산 가능.

#### 손실함수가 아닌 정확도를 지표로 할 경우 매개변수의 미분이 대부분 0이 되어 가중치 매개변수의 갱신이 멈춤